In [1]:
# Install required libraries

!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib gspread
!pip install requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 35.0 MB/s eta 0:00:00
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.173.0
    Uninstalling google-api-python-client-2.173.0:
      Successfully uninstalled google-api-python-client-2.173.0
ERROR: unknown command "install requests" - maybe you meant "install"


In [ ]:
#cell2
# Upload Gmail credentials.json file securely via Colab uploader
from google.colab import files
uploaded = files.upload()

Saving client_secret_526551064160-lg7d09fl8ohh738kkdknbsupcnvfmpmu.apps.googleusercontent.com.json to client_secret_526551064160-lg7d09fl8ohh738kkdknbsupcnvfmpmu.apps.googleusercontent.com.json


In [ ]:
#cell3
import os
import pickle
import base64
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow

SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

from google_auth_oauthlib.flow import Flow

flow = Flow.from_client_secrets_file(
    'credentials.json',
    scopes=SCOPES,
    redirect_uri='urn:ietf:wg:oauth:2.0:oob'
)
auth_url, _ = flow.authorization_url(prompt='consent')

print("Please visit this URL to authorize:")
print(auth_url)

# Paste authorization code
code = input("Enter authorization code: ")
flow.fetch_token(code=code)
creds = flow.credentials

# Finally build Gmail service object
service = build('gmail', 'v1', credentials=creds)

Please visit this URL to authorize:
https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=526551064160-lg7d09fl8ohh738kkdknbsupcnvfmpmu.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.readonly&state=FfCqfA0Pc4VWkW3sMrceGd2ZTERBXJ&prompt=consent&access_type=offline
Enter authorization code: 4/1AVMBsJjAFLezgub2ym8CHr3mpOQjsVSXg1uz9Ke8_Y4GSP_gfFvTeNYkTQk


In [ ]:
#Cell4
# Fetch recent 5 emails for processing
results = service.users().messages().list(userId='me', maxResults=5).execute()
messages = results.get('messages', [])
email_bodies = []

for msg in messages:
    txt = service.users().messages().get(userId='me', id=msg['id']).execute()
    payload = txt['payload']
    parts = payload.get("parts")

    email_text = None
    if parts:
        for part in parts:
            if part.get("mimeType") == "text/plain" or part.get("mimeType") == "text/html":
                if part.get("body") and part['body'].get('data'):
                    data = part['body'].get('data')
                    byte_code = base64.urlsafe_b64decode(data)
                    email_text = byte_code.decode("utf-8")
                    break # Found a text part, no need to check further
    elif payload.get("body") and payload['body'].get('data'):
        # Handle cases where the body is directly in the payload (less common for complex emails)
        data = payload['body'].get('data')
        byte_code = base64.urlsafe_b64decode(data)
        email_text = byte_code.decode("utf-8")


    if email_text:
        email_bodies.append(email_text)


print("✅ Successfully fetched", len(email_bodies), "emails for AI analysis.")

✅ Successfully fetched 4 emails for AI analysis.


In [ ]:
#Cell5
import google.generativeai as genai
from google.colab import userdata
import os

# Load Gemini API Key securely from Colab Secrets
os.environ["GEMINI_API_KEY"] = userdata.get('GEMINI_API_KEY')

# Configure Gemini API
genai.configure(api_key=os.environ["GEMINI_API_KEY"])

# Define the system prompt
system_prompt = """
You are an Enterprise Smart Notification Agent for campus automation.

Your task for each email:
1. Generate a 5-line summary.
2. Extract action deadlines and tasks.
3. Classify category: Academic, HR, Finance, IT, General.
4. Assign urgency: Critical, High, Normal.
5. Extract: Action required? Yes/No.

Output strictly in format:
Summary: <summary>
Category: <category>
Urgency: <urgency>
Action: <yes/no>
"""

# Load the Gemini model
model = genai.GenerativeModel(model_name="gemini-2.5-flash")

# Smart Notification Agent function
def smart_notification_agent(email_text):
    prompt = f"{system_prompt}\n\nEmail:\n{email_text}"
    response = model.generate_content(prompt)
    return response.text.strip()

# Loop over fetched emails (assume email_bodies is a list of email texts)
for idx, email_text in enumerate(email_bodies):
    print(f"\n===== Email {idx+1} =====")
    result = smart_notification_agent(email_text)
    print(result)


===== Email 1 =====
Summary: A new sign-in to your Google Account was detected. Please verify if this activity was initiated by you. If the sign-in was unauthorized, secure your account immediately. No action is needed if you recognize the sign-in. Check your account activity via the provided links.
Category: IT
Urgency: High
Action: Yes

===== Email 2 =====
Summary: This email is a LinkedIn job alert for software engineer roles.
It notifies you of 5 new job matches from Uplers in Vijayawada.
The positions include Full Stack, Backend, HubSpot Backend, Lead Data, and Senior Data Engineer.
The company is actively hiring for these roles.
You can view job details and apply through the provided links.
Category: HR
Urgency: Normal
Action: No

===== Email 3 =====
Summary: This is a LinkedIn job alert for software engineer roles.
It lists 5 new job matches in Vijayawada from Uplers.
Positions include Full Stack, Backend, Lead Data, and Senior Data Engineer.
All listed companies are actively h

In [ ]:
import gspread
from google.oauth2.service_account import Credentials

# Your service account file (already uploaded to Colab)
SERVICE_ACCOUNT_FILE = 'service_account.json'  # Replace if you used a different filename

# Define Sheets API scope
SCOPES_SHEETS = ['https://www.googleapis.com/auth/spreadsheets']

# Authenticate using Service Account
sheet_creds = Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES_SHEETS)

client = gspread.authorize(sheet_creds)

# Connect to your Google Sheet
spreadsheet_id = '1gw2yq-3ja-TdUQPh2n9xEWnlh1TEDNchOkLfKRKIBLI'  # Replace with your actual spreadsheet ID
sheet = client.open_by_key(spreadsheet_id).sheet1

# (Optional) Write header row if not already written
sheet.append_row(["Date", "Sender", "Subject", "Summary", "Category", "Urgency", "ActionRequired"])

{'spreadsheetId': '1gw2yq-3ja-TdUQPh2n9xEWnlh1TEDNchOkLfKRKIBLI',
 'updates': {'spreadsheetId': '1gw2yq-3ja-TdUQPh2n9xEWnlh1TEDNchOkLfKRKIBLI',
  'updatedRange': 'Sheet1!A1:G1',
  'updatedRows': 1,
  'updatedColumns': 7,
  'updatedCells': 7}}

In [ ]:
# Example values for testing (replace with real email content later)
# Note: In a real scenario, you would ideally get sender and subject from the fetched email metadata.
# date_today = datetime.datetime.now().strftime("%d %b") # Use full date for better tracking
# sender = "HR Dept"
# subject = "Salary Update"

import datetime # Import datetime module

# Loop through all fetched emails
for idx, email_text in enumerate(email_bodies):
    print(f"\n===== Processing Email {idx+1} =====")

    # Assuming you have fetched email metadata (like sender, subject, date) in the email fetching step
    # Replace with actual metadata from your fetched emails
    # For now, using placeholder values
    date_today = datetime.datetime.now().strftime("%Y-%m-%d") # Using YYYY-MM-DD format
    sender = f"Sender {idx+1}" # Placeholder
    subject = f"Subject {idx+1}" # Placeholder


    result = smart_notification_agent(email_text)
    print(result)

    # Parse the LLM output more robustly
    lines = result.strip().split('\n')
    summary, category, urgency, action = "", "", "", ""

    for line in lines:
        if line.startswith("Summary:"):
            summary = line.split(":", 1)[1].strip()
        elif line.startswith("Category:"):
            category = line.split(":", 1)[1].strip()
        elif line.startswith("Urgency:"):
            urgency = line.split(":", 1)[1].strip()
        elif line.startswith("Action:"):
            action = line.split(":", 1)[1].strip()

    # Insert into Google Sheet
    try:
        sheet.append_row([date_today, sender, subject, summary, category, urgency, action])
        print("✅ Successfully inserted data into Google Sheet.")
    except Exception as e:
        print(f"❌ Error inserting data into Google Sheet: {e}")


===== Processing Email 1 =====
Summary: A new sign-in was detected for your Google account. Please verify if this activity was performed by you. If the sign-in was yours, no further action is required. If it was not you, promptly check your account activity to secure it. You can review all security activities via your Google account settings.
Category: IT
Urgency: High
Action: Yes
✅ Successfully inserted data into Google Sheet.

===== Processing Email 2 =====
Summary: This LinkedIn job alert notifies you of 5 new software engineer positions.
The roles include Full Stack, Backend, HubSpot Backend, Lead Data, and Senior Data Engineer.
All opportunities are with Uplers in Vijayawada, which is actively hiring.
You can view details for each specific job via provided links.
A general link is also included to see all matching jobs on LinkedIn.
Category: HR
Urgency: Normal
Action: No
✅ Successfully inserted data into Google Sheet.

===== Processing Email 3 =====
Summary: This LinkedIn alert i